In [8]:
import pandas as pd
import numpy as np
import itertools

In [1]:
print(1+1+2)

4


In [3]:
print("hello world")

hello world


In [4]:
v = [123,32,424,324,312,341234123,412]

In [9]:
k = list(itertools.combinations(v, 2))

In [13]:
def helloworld():
    return('hello world!!!!!')

In [14]:
helloworld()

'hello world!'

In [15]:
def noworld():
    print('no world')

In [17]:
noworld()

no world
